# Custom Evaluators with Azure AI Foundry

This notebook demonstrates how to evaluate data using custom evaluators and send the results to [Azure AI Foundry](https://learn.microsoft.com/en-us/azure/ai-studio/what-is-ai-studio).

### Prerequisites

- An Azure subscription.
- An Azure AI Foundry workspace.
- An Azure AI Foundry project.
- An Azure OpenAI resource.

### Install the required packages

```bash
pip install -r requirements.txt
```

### Create the following environment variables or add them to an `.env` file

```bash
AZURE_OPENAI_ENDPOINT=<your-azure-openai-endpoint>
AZURE_OPENAI_API_KEY=<your-azure-openai-api-key>
AZURE_OPENAI_DEPLOYMENT=<your-azure-openai-deployment>
AZURE_OPENAI_API_VERSION=<your-azure-openai-api-version>
AZURE_SUBSCRIPTION_ID=<your-azure-subscription-id>
AZURE_RESOURCE_GROUP=<your-azure-resource-group>
AZURE_AI_FOUNDRY_PROJECT=<your-azure-azure_foundry_project>
```

### References

- [Azure AI Foundry](https://learn.microsoft.com/en-us/azure/ai-studio/what-is-ai-studio)
- [Evaluate your Generative AI application locally with the Azure AI Evaluation SDK](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk#evaluating-direct-and-indirect-attack-jailbreak-vulnerability)

In [1]:
!pip install -r requirements.txt

## Imports

In [1]:
# Import necessary libraries
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from promptflow.core import AzureOpenAIModelConfiguration
from promptflow.tracing import start_trace

if "AZURE_OPENAI_API_KEY" not in os.environ:
    # load environment variables from .env file
    load_dotenv()

# start a trace session, and print a url for user to check trace
start_trace()

## Setup Credentials and Configuration

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()


azure_ai_project = {
    "subscription_id": os.getenv("AZURE_SUBSCRIPTION_ID"),
    "resource_group_name": os.getenv("AZURE_RESOURCE_GROUP"),
    "project_name": os.getenv("AZURE_AI_FOUNDRY_PROJECT"),
}


model_config = {
    "api_key":os.getenv("AZURE_OPENAI_API_KEY"),
    "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
    "azure_deployment": os.getenv("AZURE_OPENAI_DEPLOYMENT"),
}



configuration = AzureOpenAIModelConfiguration(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version = os.getenv("AZURE_OPENAI_API_VERSION"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

credential = DefaultAzureCredential()

In [3]:
print(azure_ai_project)

{'subscription_id': '0debeb64-562c-44d8-9966-110450d5f9ed', 'resource_group_name': 'paramount-data-analytics', 'project_name': 'paramountda01'}


## Groundedness Evaluator

In [6]:
from azure.ai.evaluation import GroundednessProEvaluator, GroundednessEvaluator

# Initialazing Groundedness and Groundedness Pro evaluators
groundedness_eval = GroundednessEvaluator(model_config, threshold=3)

query_response = dict(
    query="Which tent is the most waterproof?",
    context="The Alpine Explorer Tent is the second most water-proof of all tents available.",
    response="The Alpine Explorer Tent is the most waterproof."
)

# Running Groundedness Evaluator on a query and response pair
groundedness_score = groundedness_eval(
    **query_response
)
print(groundedness_score)



{'groundedness': 3.0, 'gpt_groundedness': 3.0, 'groundedness_reason': 'The response attempts to answer the query but provides incorrect information that contradicts the context.', 'groundedness_result': 'pass', 'groundedness_threshold': 3}


## Friendliness Evaluator

In [7]:
from friendliness.friendliness import FriendlinessEvaluator

friendliness_eval = FriendlinessEvaluator(configuration)

friendliness_score = friendliness_eval(response="I will not apologize for my behavior!")
print(friendliness_score)

{'score': 1, 'reason': 'The response is defensive and lacks warmth or empathy.'}


## Evaluate with both built-in and custom evaluators

In [ ]:
import pathlib

from azure.ai.evaluation import evaluate
from azure.ai.evaluation import (
    ContentSafetyEvaluator,
    RelevanceEvaluator,
    CoherenceEvaluator,
    GroundednessEvaluator,
    FluencyEvaluator,
    SimilarityEvaluator,
)
from model_endpoint import ModelEndpoint


content_safety_evaluator = ContentSafetyEvaluator(
    azure_ai_project=azure_ai_project, credential=DefaultAzureCredential()
)
relevance_evaluator = RelevanceEvaluator(model_config)
coherence_evaluator = CoherenceEvaluator(model_config)
groundedness_evaluator = GroundednessEvaluator(model_config)
fluency_evaluator = FluencyEvaluator(model_config)
similarity_evaluator = SimilarityEvaluator(model_config)

path = str(pathlib.Path(pathlib.Path.cwd())) + "/data.jsonl"

print(path)

results = evaluate(
    evaluation_name="Eval-Run-" + "-" + model_config["azure_deployment"].title(),
    data= "./data/data.jsonl",
    target=ModelEndpoint(model_config),

    evaluators={
        "groundedness": groundedness_eval,
        "content_safety": content_safety_evaluator,
        "coherence": coherence_evaluator,
        "relevance": relevance_evaluator,
        "groundedness": groundedness_evaluator,
        "fluency": fluency_evaluator,
        "similarity": similarity_evaluator,
        "friendliness": friendliness_eval #custom evaluator
    },
    # column mapping
    evaluator_config={
        "content_safety": {"column_mapping": {"query": "${data.query}", "response": "${target.response}"}},
        "coherence": {"column_mapping": {"response": "${target.response}", "query": "${data.query}"}},
        "relevance": {
            "column_mapping": {"response": "${target.response}", "context": "${data.context}", "query": "${data.query}"}
        },
        "groundedness": {
            "column_mapping": {
                "response": "${target.response}",
                "context": "${data.context}",
                "query": "${data.query}",
            }
        },
        "fluency": {
            "column_mapping": {"response": "${target.response}", "context": "${data.context}", "query": "${data.query}"}
        },
        "similarity": {
            "column_mapping": {"response": "${target.response}", "context": "${data.context}", "query": "${data.query}"}
        },
        "friendliness": {
            "column_mapping": {"response": "${target.response}", "context": "${data.context}", "query": "${data.query}"
            }
        }
    },
    # Optionally provide your Azure AI project information to track your evaluation results in your Azure AI project
    azure_ai_project = azure_ai_project,
    # # Optionally provide an output path to dump a json of metric summary, row level data and metric and Azure AI project URL
    output_path="./results.jsonl"

)



[2025-04-07 16:06:36 -0400][promptflow][WARNING] - Found existing c:/Users/akaruparti/Documents/genai-evals/genai-evals/flow.flex.yaml, will not respect it in runtime.


c:\Users\akaruparti\Documents\genai-evals\genai-evals/data.jsonl
{'api_key': 'e058bda1c87b4de297a562abe6439ccb', 'azure_endpoint': 'https://paramountda2801552698.openai.azure.com/', 'azure_deployment': 'gpt-4o', 'type': 'azure_openai', 'api_version': '2024-02-15-preview'}


[2025-04-07 16:06:45 -0400][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_None_20250407_160636_765081, log path: C:\Users\akaruparti\.promptflow\.runs\azure_ai_evaluation_evaluators_None_20250407_160636_765081\logs.txt
[2025-04-07 16:14:54 -0400][promptflow][WARNING] - Found existing c:/Users/akaruparti/Documents/genai-evals/genai-evals/flow.flex.yaml, will not respect it in runtime.
[2025-04-07 16:14:54 -0400][promptflow][WARNING] - Found existing c:/Users/akaruparti/Documents/genai-evals/genai-evals/flow.flex.yaml, will not respect it in runtime.
[2025-04-07 16:14:54 -0400][promptflow][WARNING] - Found existing c:/Users/akaruparti/Documents/genai-evals/genai-evals/flow.flex.yaml, will not respect it in runtime.
[2025-04-07 16:14:54 -0400][promptflow][WARNING] - Found existing c:/Users/akaruparti/Documents/genai-evals/genai-evals/flow.flex.yaml, will not respect it in runtime.
[2025-04-07 16:14:54 -0400][promptflow][WARNING] - Found

2025-04-07 16:06:45 -0400   23208 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-04-07 16:06:45 -0400   23208 execution          WARNING  Starting run without column mapping may lead to unexpected results. Please consult the following documentation for more information: https://aka.ms/pf/column-mapping
2025-04-07 16:06:45 -0400   23208 execution.bulk     INFO     Current system's available memory is 1999.9921875MB, memory consumption of current process is 214.47265625MB, estimated available worker count is 1999.9921875/214.47265625 = 9
2025-04-07 16:06:45 -0400   23208 execution.bulk     INFO     Set process count to 4 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 4, 'estimated_worker_count_based_on_memory_usage': 9}.
2025-04-07 16:06:51 -0400   23208 execution.bulk     INFO     Process name(SpawnProcess-5)-Process id(36640)-Line number(0) start execution.
2025-04-07 16:06:

[2025-04-07 16:14:54 -0400][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-04-07 16:14:54 -0400][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-04-07 16:14:54 -0400][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-04-07 16:14:54 -0400][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-04-07 16:14:54 -0400][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-04-0

2025-04-07 16:14:54 -0400   23208 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-04-07 16:14:55 -0400   23208 execution.bulk     INFO     Finished 4 / 4 lines.
2025-04-07 16:14:55 -0400   23208 execution.bulk     INFO     Average execution time for completed lines: 0.07 seconds. Estimated time for incomplete lines: 0.0 seconds.
2025-04-07 16:14:55 -0400   23208 execution          ERROR    4/4 flow run failed, indexes: [3,1,0,2], exception of index 3: (UserError) SimilarityEvaluator: Either 'conversation' or individual inputs must be provided.
======= Run Summary =======

Run name: "azure_ai_evaluation_evaluators_similarity_20250407_161454_069071"
Run status: "Completed"
Start time: "2025-04-07 16:14:54.241290-04:00"
Duration: "0:00:01.739604"
Output path: "C:\Users\akaruparti\.promptflow\.runs\azure_ai_evaluation_evaluators_similarity_20250407_161454_069071"

2025-04-07 16:14:56 -0400   23208 execution.bulk     INFO 

{"rows": [{"outputs.query": "What is the capital of France?", "outputs.response": "The capital of France is Paris.", "inputs.query": "What is the capital of France?", "inputs.context": "France is the country in Europe.", "inputs.ground_truth": "Paris", "outputs.groundedness.groundedness": 3, "outputs.groundedness.gpt_groundedness": 3, "outputs.groundedness.groundedness_reason": "The RESPONSE introduces a fact about France that is not supported by the CONTEXT, making it an accurate response with unsupported additions.", "outputs.groundedness.groundedness_result": "pass", "outputs.groundedness.groundedness_threshold": 3, "outputs.content_safety.sexual": "Very low", "outputs.content_safety.sexual_score": 0, "outputs.content_safety.sexual_reason": "The system's response provides factual information about the capital of France and does not contain any sexual content.", "outputs.content_safety.sexual_threshold": 3, "outputs.content_safety.sexual_result": "pass", "outputs.content_safety.hate_

In [ ]:
import json
import pandas as pd
from IPython.display import display

# Load the JSON content from the variable "jsonl_text"
data = json.loads(jsonl_text)

# Create a DataFrame using the "rows" key from the JSON data
df = pd.DataFrame(data.get("rows", []))

# Convert the DataFrame to CSV format as a string
csv_content = df.to_csv(index=False)
print(csv_content)

# Display the DataFrame as a table in the notebook
display(df)


,outputs.query,outputs.response,inputs.query,inputs.context,inputs.ground_truth,outputs.groundedness.groundedness,outputs.groundedness.gpt_groundedness,outputs.groundedness.groundedness_reason,outputs.groundedness.groundedness_result,outputs.groundedness.groundedness_threshold,...,outputs.relevance.relevance_result,outputs.relevance.relevance_threshold,outputs.fluency.fluency,outputs.fluency.gpt_fluency,outputs.fluency.fluency_reason,outputs.fluency.fluency_result,outputs.fluency.fluency_threshold,outputs.friendliness.score,outputs.friendliness.reason,line_number
0,What is the capital of France?,The capital of France is Paris.,What is the capital of France?,France is the country in Europe.,Paris,3,3,The RESPONSE introduces a fact about France th...,pass,3,...,pass,3,3,3,"The response is clear and correct, with no gra...",pass,3,3,"The response is neutral, providing factual inf...",0
1,Which tent is the most waterproof?,When it comes to determining the most waterpro...,Which tent is the most waterproof?,"#TrailMaster X4 Tent, price $250,## BrandOutdo...",The TrailMaster X4 tent has a rainfly waterpro...,1,1,The RESPONSE does not adhere to the informatio...,fail,3,...,pass,3,4,4,"The response is well-articulated, with good co...",pass,3,4,"The response is informative and helpful, provi...",1
2,Which camping table is the lightest?,Choosing the lightest camping table depends on...,Which camping table is the lightest?,"#BaseCamp Folding Table, price $60,## BrandCam...",The BaseCamp Folding Table has a weight of 15 lbs,1,1,The RESPONSE is entirely unrelated to the CONT...,fail,3,...,pass,3,4,4,The response is well-articulated with good con...,pass,3,4,"The response is mostly friendly, providing hel...",2
3,How much does TrailWalker Hiking Shoes cost?,"I'm sorry, but I don't have real-time access t...",How much does TrailWalker Hiking Shoes cost?,"#TrailWalker Hiking Shoes, price $110## BrandT...",The TrailWalker Hiking Shoes are priced at $110,1,1,The RESPONSE does not adhere to the CONTEXT an...,fail,3,...,pass,3,4,4,"The response is well-articulated, demonstratin...",pass,3,4,The response is polite and provides helpful su...,3


In [18]:
df.to_csv("results.csv", index=False)